In [1]:
# !pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [2]:
import torch
import torch.nn as nn

import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
# параметры распределений
mu0, sigma0 = -2., 1.
mu1, sigma1 = 3., 2.

===========================================================================

In [4]:
# добавим еще одно измерение в генератор данных
# теперь будут два облака с центром в мат ожидании, и разбросом по дисперсии
def sample(d0, d1, n=32):
    # 1 облако
    x0_0 = d0.sample((n,))
    x0_1 = d0.sample((n,))
    x0 = torch.cat([x0_0, x0_1], 1)
    # 2 облако
    x1_0 = d1.sample((n,))
    x1_1 = d1.sample((n,))
    x1 = torch.cat([x1_0, x1_1], 1)
    
    y0 = torch.zeros((n, 1))
    y1 = torch.ones((n, 1))
    return torch.cat([x0, x1], 0), torch.cat([y0, y1], 0)

In [5]:
d0 = torch.distributions.Normal(torch.tensor([mu0]), torch.tensor([sigma0]))
d1 = torch.distributions.Normal(torch.tensor([mu1]), torch.tensor([sigma1]))

In [6]:
# внесем измениение кол-во входных параметров = 2
layer = nn.Linear(2, 1)
print([p.data[0] for p in layer.parameters()])
layer_opt = optim.SGD(lr=1e-3, params=list(layer.parameters()))

[tensor([0.3552, 0.6981]), tensor(-0.0659)]


In [7]:
%%time
log_freq = 500
for i in range(10000):
    if i%log_freq == 0:
        with torch.no_grad():
            x, y = sample(d0, d1, 100000)
            out = torch.sigmoid(layer(x))
            loss = F.binary_cross_entropy(out, y)
        print('Ошибка после %d итераций: %f' %(i/log_freq, loss))
    layer_opt.zero_grad()
    x, y = sample(d0, d1, 1024)
    out = torch.sigmoid(layer(x))
    loss = F.binary_cross_entropy(out, y)
    loss.backward()
    layer_opt.step()

Ошибка после 0 итераций: 0.126187
Ошибка после 1 итераций: 0.103830
Ошибка после 2 итераций: 0.091021
Ошибка после 3 итераций: 0.081668
Ошибка после 4 итераций: 0.074939
Ошибка после 5 итераций: 0.070154
Ошибка после 6 итераций: 0.065590
Ошибка после 7 итераций: 0.062509
Ошибка после 8 итераций: 0.060329
Ошибка после 9 итераций: 0.057359
Ошибка после 10 итераций: 0.055081
Ошибка после 11 итераций: 0.053540
Ошибка после 12 итераций: 0.051790
Ошибка после 13 итераций: 0.050821
Ошибка после 14 итераций: 0.049790
Ошибка после 15 итераций: 0.049170
Ошибка после 16 итераций: 0.047084
Ошибка после 17 итераций: 0.046567
Ошибка после 18 итераций: 0.045091
Ошибка после 19 итераций: 0.045264
CPU times: user 5.92 s, sys: 377 ms, total: 6.3 s
Wall time: 4.61 s


<b>Вопрос:</b>
Для графиков нужна проверка(тестовые данные). Для проверки мы брали функцию плотности вероятности по одному измерению. Есть ли что то подобное для нескольких измерений? Или это только ручками реализовывать?
